In [2]:
# Set up
import random
from faker import Faker
from datetime import datetime
import pymysql

fake = Faker()

# Constants
WEST_JAVA_CITIES = ['Bandung', 'Bogor', 'Sukabumi', 'Cirebon', 'Bekasi', 'Depok', 'Karawang', 'Purwakarta', 'Subang', 'Sumedang']
ITEM_TYPES = ['Electronics', 'Furniture', 'Stationary', 'Tools', 'Vehicles']

# Database connection details
DB_HOST = 'localhost'
DB_USER = 'rian'
DB_PASSWORD = 'rian'
DB_NAME = 'qti_assessment'


In [3]:
# Data generation functionss

def generate_companies(num=10):
    companies = []
    for _ in range(num):
        company = {
            'CompanyID': fake.unique.random_int(min=1, max=1000),
            'CompanyName': fake.company(),
            'CompanyLegalEntity': fake.company_suffix(),
            'City': random.choice(WEST_JAVA_CITIES)
        }
        companies.append(company)
    return companies

def generate_employees(companies, num=100):
    employees = []
    company_ids = [company['CompanyID'] for company in companies]
    for _ in range(num):
        employee = {
            'EmployeeID': fake.unique.random_int(min=1, max=1000),
            'EmployeeName': fake.name(),
            'Position': fake.job(),
            'Department': fake.bs(),
            'Division': fake.catch_phrase(),
            'CompanyID': random.choice(company_ids)  # Assign a CompanyID from the generated companies
        }
        employees.append(employee)
    return employees

def generate_supervisors(employees, num_supervisors=20):
    supervisors = []
    chosen_supervisors = set()
    chosen_employees = set()

    while len(chosen_supervisors) < num_supervisors:
        supervisor_id = random.choice(employees)['EmployeeID'] # Choosing a random employee
        if supervisor_id not in chosen_supervisors:
            chosen_supervisors.add(supervisor_id)

    for supervisor_id in chosen_supervisors: # Assigning supervised employees to supervisors
        supervised_count = random.randint(1, min(5, len(employees) - num_supervisors))
        supervised_ids = random.sample([emp['EmployeeID'] for emp in employees if emp['EmployeeID'] not in chosen_supervisors], k=supervised_count)
        for supervised_id in supervised_ids:
            if supervised_id not in chosen_employees:
                supervisors.append({
                    'SupervisorID': supervisor_id,
                    'SupervisedEmployeeID': supervised_id
                })
                chosen_employees.add(supervised_id)
    return supervisors

def generate_assets(num=500):
    assets = []
    for _ in range(num):
        asset = {
            'AssetID': fake.unique.random_int(min=1, max=1000),
            'ItemName': fake.word(),
            'ItemType': random.choice(ITEM_TYPES),
            'ItemBrand': fake.company(),
            'AssetNumber': fake.bothify(text='???-###')
        }
        assets.append(asset)
    return assets

def generate_asset_loss_reports(assets, employees, companies, num=100):
    loss_reports = []
    for _ in range(num):
        report = {
            'ReportID': fake.unique.random_int(min=1, max=1000),
            'AssetID': random.choice(assets)['AssetID'],
            'EmployeeID': random.choice(employees)['EmployeeID'],
            'CompanyID': random.choice(companies)['CompanyID'],
            'DateofReport': fake.date_this_decade()
        }
        loss_reports.append(report)
    return loss_reports

def generate_loss_events(reports, num=100):
    loss_events = []
    for _ in range(num):
        event = {
            'EventID': fake.unique.random_int(min=1, max=999999),
            'ReportID': random.choice(reports)['ReportID'],
            'Time': fake.time(),
            'Location': fake.address(),
            'EventDescription': fake.sentence()
        }
        loss_events.append(event)
    return loss_events

In [4]:
# Fucntion to connect to database using pymysql
def connect_to_db():
    connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db=DB_NAME)
    return connection

# Function to for inserting data to database
def insert_data(table_name, data, connection):
    with connection.cursor() as cursor:
        batch_size = 100 # Insert 100 records at a time
        for i, record in enumerate(data):
            placeholders = ', '.join(['%s'] * len(record))
            columns = ', '.join(f'`{col}`' for col in record.keys())
            sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            try:
                cursor.execute(sql, list(record.values()))
            except pymysql.IntegrityError as e:
                if 'Duplicate entry' in str(e) and 'PRIMARY' in str(e):
                    continue  # Skipping duplicate primary keys
                else:
                    raise  
            if (i + 1) % batch_size == 0:
                connection.commit()
                print(f"Committed batch of {batch_size} records to {table_name}.")
        connection.commit()  # Final commit


In [5]:
# Generating companies
connection = connect_to_db()
companies = generate_companies(num=35)
insert_data('company', companies, connection)

connection.close()

In [6]:
# Generating employees
employees = generate_employees(companies ,num=200)
connection = connect_to_db()
insert_data('employee', employees, connection)

connection.close()

Committed batch of 100 records to employee.
Committed batch of 100 records to employee.


In [7]:
# Generating assets
connection = connect_to_db()
assets = generate_assets(num=250)
insert_data('asset', assets, connection)

connection.close()

Committed batch of 100 records to asset.
Committed batch of 100 records to asset.


In [8]:
# Generating supervisors
connection = connect_to_db()
supervisors = generate_supervisors(employees, num_supervisors=35)
insert_data('supervisor', supervisors, connection)

connection.close()

In [9]:
# Generating loss reports
connection = connect_to_db()
loss_reports = generate_asset_loss_reports(assets, employees, companies, num=500)
insert_data('assetlossreport', loss_reports, connection)

connection.close()

Committed batch of 100 records to assetlossreport.
Committed batch of 100 records to assetlossreport.
Committed batch of 100 records to assetlossreport.
Committed batch of 100 records to assetlossreport.
Committed batch of 100 records to assetlossreport.


In [10]:
# Generating loss events
connection = connect_to_db()
loss_events = generate_loss_events(loss_reports, num=500)
insert_data('lossevent', loss_events, connection)

connection.close()

Committed batch of 100 records to lossevent.
Committed batch of 100 records to lossevent.
Committed batch of 100 records to lossevent.
Committed batch of 100 records to lossevent.
Committed batch of 100 records to lossevent.


In [11]:
# Setting up pandas
import pandas as pd

In [12]:
connection = connect_to_db()
# Select count(*) from each table
countquery = """
SELECT
    (SELECT COUNT(*) FROM employee) AS employee_count,
    (SELECT COUNT(*) FROM company) AS company_count,
    (SELECT COUNT(*) FROM asset) AS asset_count,
    (SELECT COUNT(*) FROM supervisor) AS supervisor_count,
    (SELECT COUNT(*) FROM assetlossreport) AS assetlossreport_count,
    (SELECT COUNT(*) FROM lossevent) AS lossevent_count;
"""

count_df = pd.read_sql(countquery, connection)
count_df


C:\Users\admin\AppData\Local\Temp\ipykernel_18452\3716977153.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_df = pd.read_sql(countquery, connection)


,employee_count,company_count,asset_count,supervisor_count,assetlossreport_count,lossevent_count
0,200,35,250,33,500,500


In [13]:
# Query untuk menampilkan jenis barang apa saja yang sering hilang di tiap kota
connection = connect_to_db()

mostlostitemquery = """
WITH CityLossCounts AS (
    SELECT
        C.City,
        B.ItemType,
        COUNT(*) AS LossCount,
        RANK() OVER (PARTITION BY C.City ORDER BY COUNT(*) DESC) as LossRank
    FROM AssetLossReport A
    JOIN Asset B ON A.AssetID = B.AssetID
    JOIN Company C ON A.CompanyID = C.CompanyID
    GROUP BY C.City, B.ItemType
)
SELECT City, ItemType, LossCount
FROM CityLossCounts
WHERE LossRank = 1
ORDER BY LossCount DESC;

"""

mostlostitemdf = pd.read_sql(mostlostitemquery, connection)
mostlostitemdf

C:\Users\admin\AppData\Local\Temp\ipykernel_18452\2222657233.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mostlostitemdf = pd.read_sql(mostlostitemquery, connection)


,City,ItemType,LossCount
0,Subang,Furniture,22
1,Sumedang,Tools,21
2,Bekasi,Furniture,17
3,Bogor,Stationary,16
4,Cirebon,Electronics,14
5,Depok,Furniture,14
6,Depok,Tools,14
7,Purwakarta,Stationary,11
8,Sukabumi,Tools,10
9,Bandung,Vehicles,8


In [14]:
# Query untuk menampilkan 1 atasan di setiap perusahaan dimana para staff yang dia miliki sering menghilangkan asset.
connection = connect_to_db()

supervisorquery = """
WITH SupervisorLossCounts AS (
    SELECT Sup.SupervisorID, COUNT(*) AS LossCount
    FROM AssetLossReport A
    JOIN Supervisor Sup ON A.EmployeeID = Sup.SupervisedEmployeeID
    GROUP BY Sup.SupervisorID
),
RankedSupervisors AS (
    SELECT E.CompanyID,
    E.EmployeeName AS SupervisorName, S.LossCount,
    RANK() OVER (PARTITION BY E.CompanyID ORDER BY S.LossCount DESC) as RankNumber
    FROM SupervisorLossCounts S
    JOIN Employee E ON E.EmployeeID = S.SupervisorID
)
SELECT C.CompanyName, R.SupervisorName
FROM RankedSupervisors R
JOIN Company C ON R.CompanyID = C.CompanyID
WHERE R.RankNumber = 1;
"""
supervisordf = pd.read_sql(supervisorquery, connection)
supervisordf

C:\Users\admin\AppData\Local\Temp\ipykernel_18452\124891138.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  supervisordf = pd.read_sql(supervisorquery, connection)


,CompanyName,SupervisorName
0,"Burch, Kim and Lin",Mr. Joel Perry
1,"Zhang, Wilson and Morgan",Kevin Williams
2,West LLC,Colin Lopez
3,"Valdez, Turner and Fuller",Linda Lang
4,Martin Group,Donald Garza
5,Hall Group,Christopher Frye
6,Mcmillan-King,Stephen Stanton
7,Maldonado-Sampson,Louis Duran
8,Lopez-Horton,Jasmine Small
9,Graham LLC,Linda Ruiz
